# Descripción del problema inicial

Vamos a usar el decorador con esta nueva función, introducida en la presentación.

In [1]:
import time

def mide_tiempo(funcion):
    def helper(*args, **kwargs):

        start = time.time()
        resultado = funcion(*args, **kwargs)
        total = time.time() - start
        print(total, 'segundos.')
        return resultado

    return helper

In [6]:
@mide_tiempo
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

In [4]:
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

In [7]:
fibonacci(3)

0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.


2

- ¿Porqué se repite la medición de tiempo si solo la hemos llamado una vez (`fibonacci(n)`)?

In [20]:
n = 6
print(f'El número {n} de la sucesión de fibonacci es: {fibonacci(n)}.')

0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
0.0 segundos.
El número 6 de la sucesión de fibonacci es: 8.


Este comportamiento no es inesperado. Como la función `fibonacci` es recursiva y el decorador está definido para esta, cada vez que se llama dentro de la recursión, se vuelve a llamar al decorador el cual vuelve a medir el tiempo.

Vamos a redefinirla otra vez, pero esta vez sin decorador.

In [9]:
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

Esta función con números relativamente pequeños, no tarda nada en ejecutarse.

In [10]:
n = 30
start = time.time()
print(f'El número {n} de la sucesión de fibonacci es: {fibonacci(n)}.')
print(f'{time.time() - start} segundos.')

El número 30 de la sucesión de fibonacci es: 832040.
0.13807249069213867 segundos.


Pero esta función, no crece en relación al número que le pongamos. Si le ponemos un número, más grande, tarda mucho más de lo que podemos esperar.

In [7]:
n = 40
start = time.time()
print(f'El número {n} de la sucesión de fibonacci es: {fibonacci(n)}.')
print(f'{time.time() - start} segundos.')

El número 40 de la sucesión de fibonacci es: 102334155.
16.80787181854248 segundos.


Visto el tiempo que tarda, no podremos calcular números mucho más grandes.

# Memoización

La mejor forma de optimizar el código en Python.

Hemos observado, que existen muchas dependencias en esta función. La verdadera pregunta es: ¿podemos hacer un búfer en el que ir guardando los resultados de cada vez que se calcula una nueva recursión? 

La respuesta es que sí, y lo hacemos a través de los decoradores. Podemos crear un diccionario, que actuará de caché, en el que vayamos guardando los resultados. Las claves de estos, serán los `n` y los valores, su suma correspondiente. De esta manera, no será necesario ir recorriendo cada vez el árbol hasta llegar al caso más sencillo, `n = 2` o `n = 1`, sino que miramos si `n` ya ha sido calculado, y en caso afirmativo, recogemos ese valor y no seguimos explorando.

Con ello, usaremos más memoria a costa de bajar e tiempo.

In [11]:
def memoizacion(funcion):

    cache = {}

    def helper(*args, **kargs):
        key = str(*args) + str(**kargs)

        if key not in cache:
            cache[key] = funcion(*args, **kargs)

        return cache[key]

    return helper

In [13]:
@memoizacion
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

n = 40
start = time.time()
print(f'El número {n} de la sucesión de fibonacci es: {fibonacci(n)}.')
print(f'{time.time() - start} segundos.')

El número 40 de la sucesión de fibonacci es: 102334155.
0.0 segundos.


Hemos bajado de 16 segundos, a un número de milisegundos que ni siquiera se puede detectar en el sistema. Al subirlo, el error que nos sale es porque se ha llegado al máximo de llamadas recursivas que hace de base Python. De todas formas, este límite se puede subir manualmente.

In [14]:
import sys
sys.setrecursionlimit(10_000_000)

@memoizacion
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

n = 1000
start = time.time()
print(f'El número {n} de la sucesión de fibonacci es: {fibonacci(n)}.')
print(f'{time.time() - start} segundos.')

El número 1000 de la sucesión de fibonacci es: 43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875.
0.003000497817993164 segundos.


¿Notáis el paralelismo con Streamlit? En este caso, hemos hecho uno de un diccionario que se llama `cache`. Streamlit, hace uso del mismo tipo de diccionario, que se llama `session_state`.